* Creates year-specific images, each with a band for each hazard.
* Each pixel value is the percentile value exceeded by the acrossmodel-mean hazard magnitude, from a script-specified list of percentile values.
* Percentiles are acrossmodel-means of percentiles that are calculated across the entire map, masked to include only pixels specified by the Global Human Settlement Layer as high-density or low-density urban.

In [67]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWjO0zr8CsNMyhWrrB5OJRQ7Ufmjp0hKXUv760iy57TxBbzEqDFyzQA

Successfully saved authorization token.


In [68]:
ee.Initialize()

In [69]:
hazards = ['ds', 'maxdryspell', 'mtt35', 'modlandslide']
MODELS = ['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M']
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
START_YEAR, END_YEAR = 2020, 2099
PERCENTILE_VALUES = list(range(1,100))

In [70]:
settlement = ee.ImageCollection("JRC/GHSL/P2016/SMOD_POP_GLOBE_V1")
settlement_img = settlement.filter(ee.Filter.date('2015-01-01', '2015-12-31')).first().reproject('EPSG:4326')
unsettlement_mask = settlement_img.gte(2)

In [63]:
percentiles = {h: {y: {p: 0 for p in PERCENTILE_VALUES} for y in range(START_YEAR, END_YEAR + 1)} for h in hazards}
for hazard in hazards:
    for year in range(START_YEAR, END_YEAR + 1):
        hazmags_year = ee.ImageCollection('users/tedwongwri/dataportal/magnitudes/{}'.format(hazard)).filterMetadata('year', 'equals', year)
        for model in MODELS:
            p_dict = hazmags_year.filterMetadata('model', 'equals', model).first().updateMask(unsettlement_mask).reduceRegion(ee.Reducer.percentile([75, 90, 95]), WHOLE_GLOBE).getInfo()
            for p in p_dict.keys():
                percentiles[hazard][year][int(p.split('_')[1][1:])] += int(p_dict[p]) / len(MODELS)

IndentationError: expected an indented block (<ipython-input-63-d98d167faea5>, line 8)

In [72]:
mean_factor = 1 / (len(MODELS))
for year in range(START_YEAR, END_YEAR + 1):
    all_hazards = ee.List([])
    for hazard in hazards:
        hazmags_year = ee.ImageCollection('users/tedwongwri/dataportal/magnitudes/{}'.format(hazard)).filterMetadata('year', 'equals', year)
        year_mean = ee.Image(0)
        for model in MODELS:
            year_mean = year_mean.add(hazmags_year.filterMetadata('model', 'equals', model).first().multiply(mean_factor))
        percentile_level_list = ee.List([])
        for p_idx in range(len(PERCENTILE_VALUES)):
            p = PERCENTILE_VALUES[len(PERCENTILE_VALUES) - p_idx - 1]
            percentile_level_list = percentile_levels.add(year_mean.gte(percentiles[hazard][year][p]).multiply(p))
        percentile_level = ee.ImageCollection(percentile_level_list).max()
        all_hazards = all_hazards.add(percentile_level)
    year_result = ee.ImageCollection(all_hazards).toBands().rename(['{0}_{1}'.format(h, str(year)) for h in hazards]).set('year', year)
    
    task = ee.batch.Export.image.toAsset(**{
          'image': year_result,
          'description': 'percentilelevels_{}'.format(str(year)),
          'assetId': 'users/tedwongwri/dataportal/percentilelevels/{}'.format(str(year)),
          'region': WHOLE_GLOBE,
          'crs': 'EPSG:4326',
          'dimensions': '1440x720'
        })
    task.start()
    